# SDK Reference Table - `table()` - write

Ocean Data Platform offers both API and Python SDK interfaces. This notebook highlights the Python SDK.

## Installation

If you are not working in the [ODP Workspace](https://workspace.hubocean.earth/), you need to first install the Python SDK package.

```bash
pip install -U odp-sdk
```

## Client Initialization

In [ ]:
# Import the Client class from the odp.client module
from odp.client import Client

# Create an instance of the Client class
client = Client()

If you are using our [ODP Workspaces](https://workspace.hubocean.earth/) you are automatically authenticated, but if you are working outside the initiation of the Client will open a browser to performance authentication process.
If you are working outside ODP Workspaces and you don't want to open the browser to authenticate you can use API Key authentication.
You can generate an API key in the Ocean Data Platform web interface, under your user profile.
```python
client = Client(api_key="your-api-key")
````

## Dataset Access

With an initialized `Client` you can access different datasets by using the datasets' UUID. The easiest way is to use the [ODP Catalog](https://app.hubocean.earth/catalog) to search for datasets and find the UUID (click API).

## Get Dataset

In [ ]:
# Get dataset
dataset = client.dataset("my-dataset-id") # Exchange this with your own UUID

The `dataset` from this UUID will be used in the examples below.

## Create and work with schema and table

### schema()

A key concept of the table is the schema. 

PyArrow Schema is:
- A special PyArrow object (pyarrow.lib.Schema)
- Created FROM a list of PyArrow Field objects
- Behaves LIKE a list (indexing, iteration)
- Immutable (can't be modified after creation, but the `alter()` handles this through creating a new schema and reingests the data, see below)

There is a comprehensive set of data types in PyArrow: https://arrow.apache.org/docs/python/api/datatypes.html

### create()

In order to create a table you need to have a dataset. There can only be one table so if you are trying to create a table when there is already one, you will get an error.

In [ ]:
# Import PyArrow to define schema
import pyarrow as pa

# Define a schema
schema = pa.schema([
    ('id', pa.int64()),
    ('name', pa.string()),
    ('size', pa.int64()),
    ('sensitive', pa.bool_())
])

In [ ]:
# Create a table in the dataset with the defined schema
dataset.table.create(schema)

In [ ]:
# Have a look at the table schema
print(dataset.table.schema())

In [ ]:
# Get table statistics
dataset.table.stats()

### alter()

You can do different alterations of the schema.:
- Add a column
- Drop a column
- Change name of a column
- Change type of a column

All changes are done by creating a new schema.

In [ ]:
# Get current schema to reference
old_schema = dataset.table.schema()

# Build entirely new schema from scratch
new_schema = pa.schema([
    # Copy existing fields
    old_schema.field("id"),
    
    # Add new field
    pa.field("place", pa.string()),
    
    # Copy more fields
    old_schema.field("name"),
    
    # Modify a field (change type)
    pa.field("size", pa.float64()),  # Changed from int64

    # Drop a field (not adding the field 'sensitive' to the new schema)
    
])

In [ ]:
# Apply new schema
dataset.table.alter(schema=new_schema)

In [ ]:
# Have a look at the new table schema
print(dataset.table.schema())

### drop()

This drops the table data and schema. It is **irreversible**.

In [ ]:
# Drop the table if no longer needed
dataset.table.drop()

## Working with rows in table

You insert or modify data using transactions. Key points:

- Use `with` statement - automatically handles commit/rollback
- `tx.insert(list_of_dicts)` - insert rows as list of dictionaries
- `tx.insert_batch(record_batch)` - insert PyArrow RecordBatch
- Auto-commit - on successful exit from `with` block
- Auto-rollback - if exception occurs
- All or nothing - entire transaction succeeds or fails together

Summary: Always use transactions (`with dataset.table as tx:`) to insert rows. This ensures data integrity and automatic rollback on errors!

### tx.insert()

In [ ]:
# Run this to recreate the table if you are running the notebook from top to bottom including drop() above
dataset.table.create(schema)

In [ ]:
# insert a row
with dataset.table as tx:
    tx.insert([
        {"id": 123, "place": "Sea", "name": "Kattegat", "size": 30000},
        {"id": 456, "place": "Sea", "name": "Skagerakk", "size": 32000},
        {"id": 789, "place": "Sea", "name": "Baltic", "size": 400000}
    ])

In [ ]:
# Query all data and convert to pandas DataFrame
df = dataset.table.select().all().dataframe()

In [ ]:
# Display the first few rows of the DataFrame
df.head()

,id,name,size,sensitive
0,123,Kattegat,30000,None
1,456,Skagerakk,32000,None
2,789,Baltic,400000,None


### tx.replace()

How the replace method works:
- Finds rows matching your query
- Removes them from the table
- Returns them to you (as an iterator)
- You modify them
- You re-insert them (or skip to delete)

** Note that the filter can return multiple rows and applies to all the rows. **

In [ ]:
# Replace a row

with dataset.table as tx:
    for row in tx.replace(filter="name == 'Baltic'").rows():
        # Row is removed from table at this point
        
        # Option 1: Modify and re-insert (UPDATE)
        row['size'] = 415000
        tx.insert([row])
        
        # Option 2: Don't re-insert (DELETE)
        # Just skip it
        
        # Option 3: Insert multiple rows
        #tx.insert([row, modified_row, another_row])'''

In [ ]:
# Query all data and convert to pandas DataFrame
df = dataset.table.select().all().dataframe()
# Display the first few rows of the DataFrame to see the changes
df.head()

Since the changes are applied to all rows returned, you must be careful when using.

Tips

- Replace() returns an iterator - you can loop through matching rows
- Query affects ALL matches - if 100 rows match, 100 rows are updated
- Check count first - use select() to preview
- Use specific queries - avoid accidentally updating too many rows
- Batch large updates - process in smaller transactions
- Empty query = ALL rows - be very careful!